In [ ]:
%pip install polars datasets sentencepiece transformers sentence_transformers unbabel-comet keras keras-nlp scikit-learn jax tensorflow tensorflow-text tf-keras matplotlib fasttext wget

# Set the backend

In [2]:
import os
import keras

# Disable HF Tokenizer parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Set the backend
os.environ["KERAS_BACKEND"] = "jax"

# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

# Enable mixed precision training
keras.mixed_precision.set_global_policy("mixed_float16")

# Setup training config

In [3]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    token_limit: int = 128
    dataset_limit: int = 10000
    batch_size: int = 16
    lora_rank: int = 4
    epochs: int = 10
    learning_rate: float = 2e-4
    model_id = "gemma2_instruct_2b_en"

training_config = TrainingConfig()

# Load the dataset

In [ ]:
from datasets import load_dataset
import polars as pl

ds = load_dataset("Helsinki-NLP/europarl", "cs-en")
df = ds["train"].to_polars()

# Reduce the dataset to our training limit
df = df.head(training_config.dataset_limit)

# Let's check the structure
print(df.schema)
print("\nFirst row:")
print(df.head(1))

Let's normalize the dataset

In [ ]:
# Normalize the translation column
df_norm = df.select(
    [
        pl.col("translation").struct.field("en").alias("en"),
        pl.col("translation").struct.field("cs").alias("cs"),
        pl.col("translation").struct.field("cs").str.len_chars().alias("cs_len"),
        pl.col("translation").struct.field("en").str.len_chars().alias("en_len"),
    ]
)

# Let's check the normalized structure
print(df_norm.schema)
print("\nFirst row:")
print(df_norm.head(1))

Now we need to preprocess the data to get achieve high quality results.

First, let's clean the data.

In [ ]:
def clean_texts(df):
    """
    Clean the texts by replacing multiple spaces with a single space and stripping leading and trailing spaces.
    We also filter out very short texts (less than 3 characters) and texts where one language is more than 2.5x longer than the other.
    """
    return df.with_columns(
        [
            # Clean the English text
            pl.col("en")
            .str.replace_all(r"\s+", " ")
            .str.strip_chars()
            .alias("en_clean"),
            # Clean the Czech text
            pl.col("cs")
            .str.replace_all(r"\s+", " ")
            .str.strip_chars()
            .alias("cs_clean"),
        ]
    ).filter(
        # Filter out rows with non a-Z characters
        ~pl.col("cs_clean").str.contains(r"^[a-zA-Z]+$")
        & ~pl.col("en_clean").str.contains(r"^[a-zA-Z]+$")
        # Filter out very short texts (less than 3 characters)
        & (pl.col("cs_len") >= 3)
        & (pl.col("en_len") >= 3)
        # This helps remove poor quality or misaligned translations
        & (pl.col("cs_len") / pl.col("en_len") <= 2.5)  # Czech text not too long compared to English
        & (pl.col("en_len") / pl.col("cs_len") <= 2.5)  # English text not too long compared to Czech
    )

print(f"Dataset shape: {df_norm.shape}")
df_norm = clean_texts(df_norm)
print(f"Dataset shape after cleaning: {df_norm.shape}")
print(df_norm.head(1))

Now let's run a language detection check.

In [ ]:
import fasttext
import os
import wget
from tqdm import tqdm

def detect_language(df, threshold=0.8, batch_size=100):
    """
    Detect the language of the text.
    """
    print(f"Detecting language of the text with threshold {threshold} and batch size {batch_size}")

    # Load the fasttext model
    model_path = "models/fasttext/lid.176.bin"
    if not os.path.exists(model_path):
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        wget.download("https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin", out=model_path)

    model = fasttext.load_model(model_path)

    def predict_language(text):
        """
        Check if the text is in the expected language.
        """
        prediction = model.predict(text, k=1)
        return prediction

    # Process in batches
    cs_clean = df['cs_clean'].to_list()
    en_clean = df['en_clean'].to_list()

    cs_results = []
    en_results = []
    bad_languages = []

    for i in range(0, len(cs_clean), batch_size):
        batch_cs = cs_clean[i:i+batch_size]
        batch_en = en_clean[i:i+batch_size]

        # Check if the text is in the expected language
        for text in tqdm(batch_cs, desc="Checking Czech language"):
            result = predict_language(text)
            if result[0][0] == "__label__cs":
                cs_results.append(True)
            else:
                cs_results.append(False)
                bad_languages.append(result[0][0])

        for text in tqdm(batch_en, desc="Checking English language"):
            result = predict_language(text)
            if result[0][0] == "__label__en":
                en_results.append(True)
            else:
                en_results.append(False)
                bad_languages.append(result[0][0])

    print(f"Detected bad languages: {bad_languages}")

    return df.with_columns(
        [
            pl.Series("is_valid_cs", cs_results),
            pl.Series("is_valid_en", en_results)
        ]
    ).filter(pl.col("is_valid_cs") & pl.col("is_valid_en"))

print(f"Dataset shape: {df_norm.shape}")
df_norm = detect_language(df_norm)
print(f"Dataset shape after language detection: {df_norm.shape}")
print(df_norm.head(1))

Let's filter out the rows based on similarity.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/LaBSE')

def filter_similar_texts(df, threshold=0.8, batch_size=100):
    """
    Filter out the rows based on similarity.
    """
    
    print(f"Filtering out rows with similarity less than {threshold}")
    print(f"Processing in batches of {batch_size} rows")
    
    # Process in batches
    cs_clean = df['cs_clean'].to_list()
    en_clean = df['en_clean'].to_list()
    
    similarities = []

    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for i in range(0, len(cs_clean), batch_size):
        batch_cs = cs_clean[i:i+batch_size]
        batch_en = en_clean[i:i+batch_size]
        
        # Compute the embeddings
        with torch.no_grad():
            embeddings_cs = model.encode(batch_cs, convert_to_tensor=True)
            embeddings_en = model.encode(batch_en, convert_to_tensor=True)
        
        # Compute the similarity
        batch_similarities = torch.nn.functional.cosine_similarity(embeddings_cs, embeddings_en, dim=1)
        similarities.extend(batch_similarities.cpu().numpy())
        
    return df.with_columns(pl.Series("similarity", similarities)).filter(pl.col("similarity") > threshold)

print(f"Dataset shape: {df_norm.shape}")
df_norm = filter_similar_texts(df_norm)
print(f"Filtered dataset shape: {df_norm.shape}")
print(df_norm.head(1))

Now let's perform an alignment check. We will use COMET to check the quality of the alignment.

In [ ]:
from huggingface_hub import snapshot_download
from comet import load_from_checkpoint
from tqdm import tqdm

# Load the COMET model
model_path = snapshot_download("Unbabel/wmt22-comet-da")
model_checkpoint_path = f"{model_path}/checkpoints/model.ckpt"
model = load_from_checkpoint(model_checkpoint_path)

def filter_by_quality(df, treshold = 0.4, batch_size = 100):
    """
    Filter out the rows based on the quality of the alignment.
    """
    # Use GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Get texts
    cs_clean = df["cs_clean"].to_list()
    en_clean = df["en_clean"].to_list()

    # Prepare data for COMET
    data = [
        {"src": cs, "mt": en, "ref": None}
        for cs, en in zip(cs_clean, en_clean)
    ]
    
    # Get scores from COMET
    predictions = model.predict(data, batch_size=batch_size)
    scores = predictions.scores

    return df.with_columns(
        [pl.Series("quality_score", scores)]
    ).filter(pl.col("quality_score") > treshold)


print(f"Dataset shape: {df_norm.shape}")
df_norm = filter_by_quality(df_norm)
print(f"Filtered dataset shape: {df_norm.shape}")
print(df_norm.head(10))

Now let's look at the distribution of the quality scores.

In [ ]:
# Look at distribution
print(
    df_norm.select(
        [
            pl.col("quality_score").quantile(0.25).alias("25th_percentile"),
            pl.col("quality_score").quantile(0.5).alias("median"),
            pl.col("quality_score").quantile(0.75).alias("75th_percentile"),
            pl.col("quality_score").mean().alias("mean"),
        ]
    )
)

This means that the quality of the alignment is pretty good for most of the rows.

Now let's normalize the dataset to the instruction format and save the dataset to a parquet file.

In [ ]:
def instruction_format(x):
    """Format input text into an instruction format for translation.

    Args:
        x: Dictionary containing 'en_clean' (English text) and 'cs_clean' (Czech translation)

    Returns:
        str: Formatted instruction string with the following structure:
            <start_of_turn>user
            Přelož tento text z angličtiny do češtiny.
            "{English text}"
            <end_of_turn>
            <start_of_turn>model
            {Czech translation}<end_of_turn>
    """
    return f"<start_of_turn>user\nPřelož tento text z angličtiny do češtiny.\n\"{x['en_clean']}\"<end_of_turn>\n<start_of_turn>model\n{x['cs_clean']}<end_of_turn>"

finetune_df = df_norm.with_columns(
    [
        pl.struct(["en_clean", "cs_clean"])
        .map_elements(instruction_format, return_dtype=pl.Utf8)
        .alias("instruction")
    ]
)

print(finetune_df.head(1))

No we need to tokenize the dataset. First we will load the tokenizer and test it.

In [ ]:
import keras_nlp

# Load the Gemma2 tokenizer
tokenizer = keras_nlp.models.GemmaTokenizer.from_preset(
    training_config.model_id
)

# Test the tokenizer
print(tokenizer("Hello, world!"))

Let's tokenize the dataset.

In [ ]:
print(f"Dataset shape: {finetune_df.shape}")
print(f"Token limit: {training_config.token_limit}")

finetune_df = finetune_df.with_columns(
    [
        pl.col("instruction")
        .map_elements(lambda x: tokenizer(x).tolist(), return_dtype=pl.List(pl.Int32))
        .alias("tokens")
    ]
).filter(
    pl.col("tokens").list.len() <= training_config.token_limit
)


Now let's check the token distribution.

In [ ]:
import matplotlib.pyplot as plt

token_lengths = finetune_df["tokens"].map_elements(len, return_dtype=pl.Int32)
print("Token length statistics:")
print(f"Mean: {token_lengths.mean()}")
print(f"Median: {token_lengths.median()}")
print(f"95th percentile: {token_lengths.quantile(0.95)}")
print(f"Max: {token_lengths.max()}")

# Plot the token length distribution
plt.hist(token_lengths, bins=50)
plt.title("Token Length Distribution")
plt.xlabel("Token Length")
plt.ylabel("Count")
plt.show()

Finally, filter to the token limit.

In [ ]:
finetune_df = finetune_df.select(
    [
        pl.col("instruction"),
    ]
)

print(f"Dataset shape post-tokenization: {finetune_df.shape}")
print(finetune_df.head(1))

Now finally, let's calculate the dropout rate.

In [ ]:
# Dropout rate is the percentage of rows that are filtered out.
dropout_rate = (len(df) - len(finetune_df)) / len(df)
print(f"Dropout rate: {dropout_rate:.2%}")

The dropout rate is pretty high, but it's a good thing that we have a lot of data to work with and we aim for high quality results.

Now the dataset is ready for the next step. We will use it to train the model.

In [ ]:
from sklearn.model_selection import train_test_split

# First split into train and temp (80/20)
train_df, temp_df = train_test_split(finetune_df, test_size=0.2, random_state=42)

# Then split temp into validation and test (50/50, resulting in 10/10 of original)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save all splits to parquet files
train_df.write_parquet(
    file="data/processed/gemma_cs_train.parquet", compression="zstd"
)
valid_df.write_parquet(
    file="data/processed/gemma_cs_valid.parquet", compression="zstd"
)
test_df.write_parquet(
    file="data/processed/gemma_cs_test.parquet", compression="zstd"
)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(valid_df)}")
print(f"Test size: {len(test_df)}")


# Load the model

In [ ]:
import keras
import keras_nlp

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(training_config.model_id)
gemma_lm.summary()

Let's test the inference of our prompt.

In [ ]:
prompt = 'Přelož tento text z angličtiny do češtiny:\n"Hello, world!"'
print(
    gemma_lm.generate(
        prompt,
        max_length=training_config.token_limit,
    )
)

# LoRA Fine-tuning

In [ ]:
# Enable LoRA on the model
gemma_lm.backbone.enable_lora(rank=training_config.lora_rank)
gemma_lm.summary()

# Control memory usage
gemma_lm.preprocessor.sequence_length = training_config.token_limit

# Setup optimizer
optimizer = keras.optimizers.AdamW(
    learning_rate=training_config.learning_rate,
    weight_decay=0.01,
    epsilon=1e-6, # Default is 1e-8, but we need to use mixed precision training
)

# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# Setup loss
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Setup metrics
metrics = [keras.metrics.SparseCategoricalAccuracy()]

# Compile the model
gemma_lm.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Reduce the dataset to Keras expected format


In [ ]:
train_dataset = train_df["instruction"].to_list()
valid_dataset = valid_df["instruction"].to_list()

print(f"Train dataset shape: {len(train_dataset)}")
print(train_dataset[0])
print("\n")

print(f"Valid dataset shape: {len(valid_dataset)}")
print(valid_dataset[0])

The model is now ready for training. Let's train it.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from wandb.integration.keras import WandbMetricsLogger
import wandb

class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch} finished")
        model_name = f"gemma_cs_{training_config.lora_rank}_epoch{epoch+1}.lora.h5"
        gemma_lm.backbone.save_lora_weights(model_name)
        print(f"Model {model_name} saved")

        # Eval on prompt
        prompt = "Přelož tento text z angličtiny do češtiny.\n\"Hello, world!\""
        print(f"Prompt: {prompt}")
        print(
            f"Response: {gemma_lm.generate(prompt, max_length=training_config.token_limit)}"
        )

# Initialize a new W&B run
wandb.init(config={
    "learning_rate": training_config.learning_rate,
    "epochs": training_config.epochs,
    "batch_size": training_config.batch_size,
    "lora_rank": training_config.lora_rank,
    "token_limit": training_config.token_limit,
    "model": "gemma2_instruct_2b_en",
    "task": "czech-context"
})

# Convert lists to TensorFlow datasets
train_ds = tf.data.Dataset.from_tensor_slices(train_dataset)
valid_ds = tf.data.Dataset.from_tensor_slices(valid_dataset)

# Configure datasets for performance
train_ds = train_ds.batch(training_config.batch_size)
valid_ds = valid_ds.batch(training_config.batch_size)

# Enable prefetching to improve performance
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

# Train the model
history = gemma_lm.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=training_config.epochs,
    callbacks=[CustomCallback(), WandbMetricsLogger()],
)

# Plot the training loss
plt.plot(history.history["loss"])
plt.show()

# Plot the accuracy
plt.plot(history.history["sparse_categorical_accuracy"])
plt.show()